In [1]:
import pandas as pd

In [2]:
df = df = pd.read_csv("../../dados_churn.csv")
df.head()

,matricula,plano_id,frequencia_semanal,tempo_ultimo_checkin,duracao_media,churn
0,1,1,0.880952,0,113.333333,0
1,2,2,0.244444,17,109.090909,0
2,3,1,0.411765,13,120.000000,0
3,4,3,1.245614,3,125.070423,0
4,5,2,0.589041,1,124.186047,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1996 entries, 0 to 1995
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   matricula             1996 non-null   int64  
 1   plano_id              1996 non-null   int64  
 2   frequencia_semanal    1996 non-null   float64
 3   tempo_ultimo_checkin  1996 non-null   int64  
 4   duracao_media         1996 non-null   float64
 5   churn                 1996 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 93.7 KB


In [4]:
df.describe()

,matricula,plano_id,frequencia_semanal,tempo_ultimo_checkin,duracao_media,churn
count,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000
mean,1000.817635,2.032565,2.379908,101.793086,119.530465,0.502505
std,578.285037,0.816154,6.686139,142.754667,10.583106,0.500119
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,499.750000,1.000000,0.432219,4.000000,114.000000,0.000000
50%,1000.500000,2.000000,0.785330,28.500000,120.000000,1.000000
75%,1501.250000,3.000000,1.600000,152.250000,125.454545,1.000000
max,2003.000000,3.000000,74.000000,789.000000,168.000000,1.000000


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import make_scorer
import numpy as np

# Remover 'churn' e 'matricula' de X
X = df.drop(['churn', 'matricula'], axis=1)  # Agora removemos 'matricula' também
y = df['churn']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o modelo
model = GradientBoostingClassifier()

# Definir o espaço de parâmetros para o Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

# Configurar o GridSearchCV com 10-fold cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=10, scoring='accuracy', verbose=1)

# Ajustar o modelo com os dados de treinamento
grid_search.fit(X_train, y_train)

# Melhor conjunto de parâmetros
print(f"Best parameters: {grid_search.best_params_}")

# Avaliar o desempenho com o melhor modelo encontrado
best_model = grid_search.best_estimator_

# Realizar a validação cruzada para obter uma melhor média de desempenho
cross_val_score_result = cross_val_score(best_model, X_train, y_train, cv=10, scoring='accuracy')
print(f"Cross-validation scores: {cross_val_score_result}")
print(f"Mean cross-validation score: {np.mean(cross_val_score_result):.4f}")

# Treinamento do modelo final com os dados completos
best_model.fit(X_train, y_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
Cross-validation scores: [0.98125    0.9375     0.94375    0.95       0.90625    0.95
 0.95597484 0.94339623 0.93081761 0.94968553]
Mean cross-validation score: 0.9449


GradientBoostingClassifier(learning_rate=0.05, n_estimators=200)

In [7]:
import joblib

# Supondo que seu modelo treinado seja o `best_model`
joblib.dump(best_model, 'churn_model.pkl')

['churn_model.pkl']